In [ ]:
import numpy as np
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import pandas as pd
import seaborn as sn
import os

## Pretrained ResNet-50 Model for extracting Features

In [ ]:
resnet50 = models.resnet50(pretrained=True)

In [ ]:
resnet50

#### Changes in the model architecture easy to capture the feature from tha image datasets

We have removed the by default classifier and added one output layer to classify the binary output.

In [ ]:
# Add the new layer after avgpool
class ModifiedResNet(nn.Module):
    def __init__(self, original_model):
        super(ModifiedResNet, self).__init__()
        self.features = nn.Sequential(
            *list(original_model.children())[:-2],  # All layers up to avgpool
            original_model.avgpool 
        )
        self.fc = nn.Sequential(nn.Linear(original_model.fc.in_features, 512),
                                nn.ReLU())
        self.out = nn.Linear(512, 2)  # Original FC layer

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  # Flatten for FC layers
        x = self.fc(x)    # Pass through the new layer
        x = self.out(x)           # Pass through the original FC layer
        return x

In [ ]:
model = ModifiedResNet(resnet50)

In [ ]:
model

## Model Training for feature extraction

Uses the combined Dataframe from two datasets (DDR & APTOS)

In [ ]:
df = pd.read_csv('final_merged.csv')
df

### A pytorch custom dataset that takes the combined CSV file, all-together image folder, & totensor() transfromation as input and output image, their label & image file name 

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        super().__init__()
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        image_file = os.path.join(self.root_dir, self.df.iloc[index, 0]+".jpg")
        image = Image.open(image_file).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.df.iloc[index, 1], dtype=torch.long)
        return image, label, image_file

In [ ]:
# Define transformation pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),       # Resize to 224x224
    transforms.ToTensor(),               # Convert image to PyTorch tensor
])
# Instantiate the dataset
csv_file = 'final_merged.csv' 
root_dir = 'data/merged_binary/'
dataset = CustomDataset(csv_file=csv_file, root_dir=root_dir, transform=transform)

### Dataloader

In [ ]:
# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Training part of the model

In [ ]:
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    print("epoch-------------------------->", epoch)
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        # Zero the gradient buffers
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        print("Loss: ", loss.item())
    train_accuracy = 100.0 * correct / total

In [ ]:
train_accuracy

In [ ]:
torch.save(model.state_dict(), "fully_trained.pth")

## Feature Extraction from the Trained model

We have dropped the final classifier layer from the trained model and get output from one earlier layer that outputs 512 features per sample

In [ ]:
model = ModifiedResNet(resnet50)
model.load_state_dict(torch.load("fully_trained.pth"))
model.out = nn.Identity()
model.eval()

In [ ]:
features_list = []
filenames = []
labels = []

with torch.no_grad():  # No need to compute gradients
    for inputs, targets, file_name in dataloader:
        # Extract features
        inputs = inputs.to('cpu')  # Move to GPU if available 
        print(inputs.shape)
        outputs = model(inputs).squeeze()
        print(outputs.shape)
        # # Append features and metadata
        features_list.append(outputs.numpy())  # Convert to NumPy array
        filenames.extend(file_name)
        labels.extend(targets.numpy())

In [ ]:
# Combine features into a single array
features_array = np.vstack(features_list)

In [ ]:
features_array.shape

### Convert the (n_samples, 512 features) to Dataframe

In [ ]:
df = pd.DataFrame(features_array)
df['file_name'] = filenames
df['label'] = labels

In [ ]:
df

In [ ]:
df.to_csv("features_512.csv", index=False)